# Chat with vision models

**If you're looking for the web application, check the src/ folder.**

This notebook is just provided for manual experimentation with the vision model.

## Authenticate to OpenAI

The following code connects to OpenAI, either using an Azure OpenAI account, GitHub models, or local Ollama model. See the README for instruction on configuring the `.env` file.

In [6]:
import os

import azure.identity
import openai
from dotenv import load_dotenv

load_dotenv(".env", override=True)

openai_host = os.getenv("OPENAI_HOST", "github")

if openai_host == "github":
    print("Using GitHub Models with GITHUB_TOKEN as key")
    openai_client = openai.OpenAI(
        api_key=os.environ["GITHUB_TOKEN"],
        base_url="https://models.github.ai/inference",
    )
    model_name = os.getenv("OPENAI_MODEL", "openai/gpt-4o")
elif openai_host == "local":
    print("Using local OpenAI-compatible API with no key")
    openai_client = openai.OpenAI(api_key="no-key-required", base_url=os.environ["LOCAL_OPENAI_ENDPOINT"])
    model_name = os.getenv("OPENAI_MODEL", "gpt-4o")
elif openai_host == "azure" and os.getenv("AZURE_OPENAI_KEY_FOR_CHATVISION"):
    # Authenticate using an Azure OpenAI API key
    # This is generally discouraged, but is provided as a convenience
    print("Using Azure OpenAI with key")
    openai_client = openai.OpenAI(
        base_url=os.environ["AZURE_OPENAI_ENDPOINT"] + "/openai/v1/",
        api_key=os.environ["AZURE_OPENAI_KEY_FOR_CHATVISION"],
    )
    # This is actually the deployment name, not the model name
    model_name = os.getenv("OPENAI_MODEL", "gpt-4o")
elif openai_host == "azure" and os.getenv("AZURE_OPENAI_ENDPOINT"):
    tenant_id = os.environ["AZURE_TENANT_ID"]
    print("Using Azure OpenAI with Azure Developer CLI credential for tenant id", tenant_id)
    default_credential = azure.identity.AzureDeveloperCliCredential(tenant_id=tenant_id)
    token_provider = azure.identity.get_bearer_token_provider(
        default_credential, "https://cognitiveservices.azure.com/.default"
    )
    openai_client = openai.OpenAI(
        base_url=os.environ["AZURE_OPENAI_ENDPOINT"] + "/openai/v1/",
        api_key=token_provider,
    )
    # This is actually the deployment name, not the model name
    model_name = os.getenv("OPENAI_MODEL", "gpt-4o")

print(f"Using model {model_name}")

Using GitHub Models with GITHUB_TOKEN as key
Using model openai/gpt-4o


## Send an image by URL

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {"text": "Is this a unicorn?", "type": "text"},
            {
                "image_url": {"url": "https://upload.wikimedia.org/wikipedia/commons/6/6e/Ur-painting.jpg"},
                "type": "image_url",
            },
        ],
    }
]
response = openai_client.chat.completions.create(model=model_name, messages=messages, temperature=0.5)

print(response.choices[0].message.content)

No, this is not a unicorn. This is an illustration of an aurochs, an extinct species of large wild cattle that once roamed Europe, Asia, and North Africa. Unicorns are mythical creatures typically depicted with a single horn on their forehead, while this animal clearly has two horns.


## Send an image by Data URI



In [8]:
import base64


def open_image_as_base64(filename):
    with open(filename, "rb") as image_file:
        image_data = image_file.read()
    image_base64 = base64.b64encode(image_data).decode("utf-8")
    return f"data:image/png;base64,{image_base64}"

In [ ]:
response = openai_client.chat.completions.create(
    model=model_name,
    messages=[
        {
            "role": "user",
            "content": [
                {"text": "are these alligators or crocodiles?", "type": "text"},
                {"image_url": {"url": open_image_as_base64("mystery_reptile.png")}, "type": "image_url"},
            ],
        }
    ],
)

print(response.choices[0].message.content)

These are crocodiles. You can tell by their slender, V-shaped snouts, which are a characteristic feature of crocodiles, as opposed to the broader, U-shaped snouts of alligators.


## Use cases for image analysis

### Accessibility

#### Assistance for vision-impaired

In [11]:
response = openai_client.chat.completions.create(
    model=model_name,
    messages=[
        {
            "role": "user",
            "content": [
                {"text": "is there anything good for vegans on this menu?", "type": "text"},
                {"image_url": {"url": open_image_as_base64("menu.png")}, "type": "image_url"},
            ],
        }
    ],
)

print(response.choices[0].message.content)

There are limited options for vegans on this menu, as many dishes include animal-based ingredients like cheese, cream, meat, and seafood. However, a few dishes could potentially be adapted to suit a vegan diet. Here's a breakdown:

### Possible Vegan/Adaptable Options:
1. **Spinaci Soffritti** (Antipasti) - Fresh spinach sautéed with lemon and garlic. This dish appears to be naturally vegan.
   
2. **Insalata Di Maria** (Zuppe & Insalate) - Seasonal greens, farro salad, cherry tomatoes, and avocado, tossed with Carmine’s House Vinaigrette. Double-check the vinaigrette to ensure it doesn't contain honey or other non-vegan ingredients.

3. **Panzenella con Fagioli** (Zuppe & Insalate) - Vine tomatoes, bread salad, onions, cannellini beans, cucumbers, and avocado, tossed with Carmine’s House Vinaigrette. Check whether the bread is made without dairy or eggs.

4. **Spaghetti Ortolano** (Pasta & Risotto) - Arugula, tomatoes, pine nuts, and extra virgin olive oil. Omit the goat cheese to mak

#### Automated image captioning

In [ ]:
response = openai_client.chat.completions.create(
    model=model_name,
    messages=[
        {
            "role": "user",
            "content": [
                {"text": "Suggest an alt text for this image", "type": "text"},
                {"image_url": {"url": open_image_as_base64("azure_arch.png")}, "type": "image_url"},
            ],
        }
    ],
)

print(response.choices[0].message.content)

Diagram showing the architecture of a containerized chat application in Azure. Central to the design is a container app, labeled "chat-app-cnbr6e7fwgzj2-ca," deployed within a "Container Apps Environment." The app connects to Azure services, including "Azure AI services (cnbr6e7fwgzj2-cog)" and a "Managed Identity (chat-app-cnbr6e7fwgzj2-id-aca)." Supporting components include a "Log Analytics workspace (chat-app-cnbr6e7fwgzj2-log)," a "Container registry (chatappcnbr6e7fwgzj2registry)," and a "Key vault (chatappcnbr6e7f-vault)." Interactions are indicated by arrows connecting the components, representing integration and workflow.


### Business process automation

#### Insurance claim processing

In [13]:
response = openai_client.chat.completions.create(
    model=model_name,
    messages=[
        {
            "role": "system",
            "content": (
                "You are an AI assistant that helps auto insurance companies process claims."
                "You accept images of damaged cars that are submitted with claims, and you are able to make judgments "
                "about the causes of automobile damage, and the validity of claims regarding that damage."
            ),
        },
        {
            "role": "user",
            "content": [
                {"text": "Claim states that this damage is due to hail. Is it valid?", "type": "text"},
                {"image_url": {"url": open_image_as_base64("dented_car.jpg")}, "type": "image_url"},
            ],
        },
    ],
)

print(response.choices[0].message.content)

This damage does not appear to be caused by hail. Hail damage typically consists of numerous small dents or dings spread across the surface of the vehicle, such as the hood, roof, and trunk, and not extensive impact damage that transforms or crushes the structure as shown in this image. 

The claim stating this damage is hail-related is likely invalid, as it more closely resembles damage caused by a collision or impact with another object, such as another vehicle or a stationary barrier.


#### Graph analysis

In [15]:
messages = [
    {
        "role": "user",
        "content": [
            {"text": "What zone are we losing the most trees in?", "type": "text"},
            {
                "image_url": {
                    "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/1/1f/20210331_Global_tree_cover_loss_-_World_Resources_Institute.svg/1280px-20210331_Global_tree_cover_loss_-_World_Resources_Institute.svg.png"
                },
                "type": "image_url",
            },
        ],
    }
]
response = openai_client.chat.completions.create(model=model_name, messages=messages, temperature=0.5)

print(response.choices[0].message.content)

The zone where we are losing the most trees is the **Tropical zone**, represented by the dark green bars. It consistently shows the largest annual loss in tree cover compared to the other zones (Boreal, Temperate, and Subtropical).


#### Table analysis

In [16]:
response = openai_client.chat.completions.create(
    model=model_name,
    messages=[
        {
            "role": "user",
            "content": [
                {"text": "What's the cheapest plant?", "type": "text"},
                {"image_url": {"url": open_image_as_base64("page_0.png")}, "type": "image_url"},
            ],
        }
    ],
)

print(response.choices[0].message.content)

The cheapest plant available is **Thringrass (Agrostis pallens)**, priced at **$0.58** per stub.


#### Appliance support

In [ ]:
response = openai_client.chat.completions.create(
    model=model_name,
    messages=[
        {
            "role": "user",
            "content": [
                {"text": "How do I set this to wash the dishes quickly?", "type": "text"},
                {"image_url": {"url": open_image_as_base64("dishwasher.png")}, "type": "image_url"},
            ],
        }
    ],
)

print(response.choices[0].message.content)

To wash the dishes quickly with this Bosch dishwasher, follow these steps:

1. **Turn on the dishwasher**: Press the **"On/Off"** button.
2. **Select the quick program**: Press the **"Quick 45°"** button. This is a designated fast wash program, typically for lightly soiled dishes.
3. **Optional - Use VarioSpeed**: If you want to make the cycle even faster, press the **"VarioSpeed"** button, which reduces the cycle time further by increasing energy and water usage.
4. **Start the dishwasher**: Press the **"Start"** button to begin the cycle.

That's it! Your dishwasher will now wash the dishes quickly. Keep in mind that this setting is best suited for lightly soiled dishes and not for heavy loads or tough stains.
